# Example:  Gauss Histogram Fitting 

This example shows how to fit an histogram using ROOT 

Create First an empty histogram with 50 bins with range [-10,10]

In [ ]:
TH1D h1("h1","h1",50,-8,8);

Fill the histogram with 10000 Gaussian Random number with mean=1 and sigma=2

In [ ]:
for (int i = 0; i < 1000; ++i) {
    h1.Fill(gRandom->Gaus(1,2));
}

Note, we could also use the function h1.FillRandom("gaus"), but we need to set the right parameters of the Gaussian function before.

In [ ]:
%jsroot on
h1.DrawCopy();
gPad->Draw();

### Part1: Fit the Histogram with a Gaussian function

Note that before fitting we need to create the fitting function and set its initial parameter values. 
For a Gaussian function the setting of initial values is not really needed, because they can be computed automatically by ROOT. 

In [ ]:
// declare here some variables which will be used afterwards
TF1 * f1 = nullptr; 
TFitResultPtr res; 

Create the Gaussian Fit function

In [ ]:
f1 = new TF1("f1","gaus",-8,8);

Set the initial parametger values (e.g. Constant = 100, mean = 0, sigma =1)

In [ ]:
f1->SetParameters(100,0,1);  

Fit now the histogram using the Fit method in ROOT. By default the least-square method is used. 
For likelihood fits we need to use the option "L", for Pearson chi-square (expected error) , option "P".

Use also  option "S" to create a TFitResult object that is returned to the user.
To compute the error using MINOS, use the "E" option


In [ ]:
res  = h1.Fit(f1, "S  E");

In [ ]:
gStyle->SetOptFit(1111);
gPad->SetLogy(true); // to set log scale in y
gPad->Draw();

Print the result of the Fit from the returned TFitResult object

In [ ]:
res->Print();

We can plot the fit residuals using , see https://root.cern.ch/doc/master/classTRatioPlot.html 

In [ ]:
auto rp1 = new TRatioPlot(&h1);
rp1->Draw();
rp1->GetLowerRefYaxis()->SetTitle("ratio");
rp1->GetUpperRefYaxis()->SetTitle("entries");
gPad->Draw();

We can print also the Chisquare value of the fit. 
- we print Chi2 computed using the bin values with observed errors (Neyman Chi2)
- we print also the Chi2 obtained from the Poisson likeihood (Baker-Cousins Chi2) 

note that in the first case the empty bins are not used, while in the second one all bins are used including empty ones. 

In [ ]:
std::cout << "Neyman chi2             = " << h1.Chisquare(f1) << "/ NDF = " << f1->GetNDF() << std::endl;
std::cout << "Poisson likelihood chi2 = " << h1.Chisquare(f1,"L") << "/ NDF = " << h1.GetNbinsX()-f1->GetNpar() << std::endl;

Get the correlation matrix of the fit from the TFitResult class and print it 

In [ ]:
auto corrMatrix = res->GetCorrelationMatrix(); 
corrMatrix.Print(); 

In [ ]:
std::cout  << "Gaussian sigma = " << f1->GetParameter("Sigma") << "  +/- " << f1->GetParError(f1->GetParNumber("Sigma")) << std::endl;

If we want to access the MINOS asymmetric error, we can get them from the FitResult object

In [ ]:
std::cout  << "Gaussian sigma = " << res->Parameter(2) << "  + " << res->UpperError(2) << "  " << res->LowerError(2) << std::endl;

### Part 2:  Fit Bias Study

We study now the Fit Bias for the sigma parameter. We can look at the difference obtained by using 3 different fit methods: 

1. Chisquare fit using the observed error (default case in ROOT) 
2. Chisquare fit using the expected error (option "P" - Pearson chi2)
3. Binned maximum likelihood fit (option "L")

In [ ]:
TH1 * hs = nullptr;

In [ ]:
hs = new TH1D("hs","Sigma pull distribution",50,-4,4);

Generate 1000 pseudo-experiments, where for each one of them generate an histogram as above (50 bins, in [-10,10] and fill it with 1000 Gaussian distributed numbers with $\mu=1$ and $\sigma=2$. 
For each pseudo-experiment, fit the histogram and look at the obtained $\sigma$ value from the fit. 
Fill the *Sigma pull histogram*, *hs* with $(\sigma_{FIT} - \sigma_{TRUE})/\sigma_{FIT-ERROR}$. 

$\sigma_{TRUE} = 2$ in this case and $\sigma_{FIT-ERROR}$ is the uncertainty obtained from the fit.

Do for one of the three cases (Neyman chi2 (default), Likelihood fit (option *L*) or Pearson chi2 (option *P*)

In [ ]:
hs->Reset(); // in case we run a second time
for (int iexp = 0; iexp < 1000; ++iexp) { 
    TH1D h0("h0","h0",50,-8,8);
    for (int i = 0; i < 1000; ++i){
        h0.Fill(gRandom->Gaus(0,2));
    }
    TF1 f1("f1","gaus",-8,8);
    f1.SetParameters(100,0,1);
    h0.Fit(&f1,"Q  P");  // use Q option to avoid too much printing
    hs->Fill( (f1.GetParameter(2)- 2)/f1.GetParError(2));  // to get sigma and its error
    //std::cout << f1.GetParameter(2) << " " << f1.GetParError(2) << std::endl;
}

Fit the obtained Pull distribution with a Gaussian function

In [ ]:
hs->Fit("gaus"); gPad->Draw();

### Part 3:  Study the chi2 distribution

Using the likelihood definition described in the Baker-Cousins paper we can use the likelihood at the minimum as a chi2. We study its distribution using pseudo-experiments.
We can compare what we obtain if we use the likelihood-value in a likelihood fit or the chi2 obtained when doing a
chi2 Fit (Neyman or Pearson chi2). 

In [ ]:
TH1 * hchi = nullptr;
TF1 * fchi = nullptr; 

In [ ]:
hchi = new TH1D("hchi","chi-squared distribution",100,0,100);

The Baker-Cousins likelihood-value is obtained from the FitResult class as 
$2 \times$ *result->MinFcnValue()*
*result->Chi2()* returns instead the Chi2 obtained from the data-function resuduals.

Genersate now 10000 pseudo-experiments and for each of them create, fill and fit an histogram as before. 
But now make an histogram of the chi2 distribution, either using *2 result->MinFcnValue()* if doing a likelihood fit or
using *result->Chi2()* in case of a chi2 fit

In [ ]:
hchi->Reset(); // in case we run a second time
for (int iexp = 0; iexp < 10000; ++iexp) { 
    TH1D h0("h0","h0",50,-8,8);
    for (int i = 0; i < 1000; ++i){
        h0.Fill(gRandom->Gaus(0,4));
    }
    TF1 f1("f1","gaus",-8,8);
    f1.SetParameters(100,0,1);
    // for likelihood fits 
    auto r = h0.Fit(&f1,"S Q L ");  // use Q option to avoid too much printing
    hchi->Fill( 2.* r->MinFcnValue() );  // to get likelihood value from fit result
    // uncomment these lines for chi2 fits and comment the two lines above
    //auto r = h0.Fit(&f1,"S Q ");  // use Q option to avoid too much printing
    //hchi->Fill( r->Chi2 () );  //  get chi2 value from fit result
}

In [ ]:
hchi->Draw(); gPad->Draw();

In [ ]:
fchi = new TF1("fchi","[0]*ROOT::Math::chisquared_pdf(x,[1])",0,100);
fchi->SetParameters(hchi->GetEntries()*hchi->GetBinWidth(1), hchi->GetMean());

In [ ]:
hchi->Fit(fchi,"L");

In [ ]:
hchi->Draw(); gPad->Draw();

Which of the two obtained quantities (Baker-Cousins lieklihood or Neyman chi2) agree better with a real chi2 distribution ? 